<a href="https://colab.research.google.com/github/eman-kom/tensorflow-fundamentals/blob/main/08_NLP_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-6f537a20-6d28-889a-55dd-4f72c9276d13)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-04 09:49:57--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-04 09:49:57 (117 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2022-07-04 09:50:00--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.148.128, 74.125.124.128, 172.253.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.148.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.003s  

2022-07-04 09:50:00 (166 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "<real disaster>" if target > 0 else "<not a real disaster>")
  print(f"Text\n{text}\n")
  print("---\n")

Target: 0 <not a real disaster>
Text
@MaatMHI Slightly diff catastrophe &amp; Barry was running solo but generally the same thing.

---

Target: 1 <real disaster>
Text
Strong Thunderstorm 4 Miles East of Pickens Moving NE At 20 MPH. Pea Size Hail and Wind Gusts Up to 40 MPH... #scwx http://t.co/TsaLetFtkt

---

Target: 0 <not a real disaster>
Text
Edwin wow. Crushed.

---

Target: 1 <real disaster>
Text
Chick masturbates a guy until she gets exploded on her face &gt; http://t.co/5QhoeHE9hf

---

Target: 0 <not a real disaster>
Text
i lava you! ????

---



In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,
    random_state=42
)

In [11]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [12]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens=10000,
    output_mode="int",
    output_sequence_length=15,
)

In [14]:
text_vectorizer.adapt(train_sentences)

In [15]:
text_vectorizer(["There's a flood in my street!"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [16]:
random_sent = random.choice(train_sentences)
print(f"Before: {random_sent}")
text_vectorizer([random_sent])

Before: Rly tragedy in MP: Some live to recount horror: ÛÏWhen I saw coaches of my train plunging into water I called ... http://t.co/21hsrrqZOu


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 922,  360,    4,  599,   82,  199,    5, 1063,  650, 1158,    8,
         414,  860,    6,   13]])>

In [17]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [18]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=10000,
    output_dim=128,
    input_length=15
)

embedding

In [19]:
sample_embed = embedding(text_vectorizer([random.choice(train_sentences)]))
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01963902,  0.03132067, -0.02268289, ...,  0.0041335 ,
         -0.012157  , -0.03662957],
        [ 0.02445105,  0.02771156, -0.04110926, ...,  0.0351442 ,
          0.0241346 , -0.00152315],
        [ 0.01952514,  0.00574758, -0.02844809, ...,  0.01583779,
          0.00789567,  0.01370254],
        ...,
        [ 0.01878253,  0.00147899, -0.02684787, ...,  0.01670672,
          0.00772765,  0.02174525],
        [ 0.01952514,  0.00574758, -0.02844809, ...,  0.01583779,
          0.00789567,  0.01370254],
        [ 0.01966728, -0.01331524,  0.03681232, ...,  0.02472403,
         -0.01517858,  0.02176131]]], dtype=float32)>

# Model 0: Baseline Model -> Sklearn Multinomial Naive Bayes using TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [21]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline Score: {baseline_score*100:.2f}%")

Baseline Score: 79.27%


In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [23]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:5]

array([1, 1, 1, 0, 0])

In [24]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [25]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

# Model 1: Simple Dense Model

In [26]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [27]:
model_1.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [28]:
model_1_history = model_1.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name="model_1_dense")]
)

Saving TensorBoard log files to: model_logs/model_1_dense/20220704-095007
Epoch 1/5
215/215 [==============================] - 8s 10ms/step - loss: 0.6111 - accuracy: 0.6878 - val_loss: 0.5334 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4429 - accuracy: 0.8186 - val_loss: 0.4685 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3481 - accuracy: 0.8597 - val_loss: 0.4574 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2849 - accuracy: 0.8908 - val_loss: 0.4635 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2374 - accuracy: 0.9111 - val_loss: 0.4813 - val_accuracy: 0.7900


In [29]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4813 - accuracy: 0.7900


[0.48129788041114807, 0.7900262475013733]

In [30]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

array([[3.63508672e-01],
       [6.98214769e-01],
       [9.97214615e-01],
       [8.57766941e-02],
       [1.26392856e-01],
       [9.44538593e-01],
       [9.14806604e-01],
       [9.92478371e-01],
       [9.65327024e-01],
       [2.05557674e-01],
       [1.00252040e-01],
       [6.29545987e-01],
       [5.40799983e-02],
       [2.07237959e-01],
       [3.64532368e-03],
       [1.22955188e-01],
       [2.20040493e-02],
       [8.52999911e-02],
       [2.08593398e-01],
       [4.80998933e-01],
       [9.10491347e-01],
       [3.28270309e-02],
       [3.65864694e-01],
       [7.52383694e-02],
       [9.52058554e-01],
       [9.98812675e-01],
       [2.75654700e-02],
       [6.50133342e-02],
       [2.51512341e-02],
       [1.68594435e-01],
       [5.08424640e-01],
       [2.00960979e-01],
       [4.67826903e-01],
       [1.87610477e-01],
       [5.07441401e-01],
       [4.98532504e-02],
       [9.92858648e-01],
       [1.17087886e-01],
       [3.12726796e-02],
       [9.98651087e-01],


In [31]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [32]:
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results, baseline_results

({'accuracy': 79.00262467191601,
  'f1': 0.7869273549752186,
  'precision': 0.7955103864713003,
  'recall': 0.7900262467191601},
 {'accuracy': 79.26509186351706,
  'f1': 0.7862189758049549,
  'precision': 0.8111390004213173,
  'recall': 0.7926509186351706})

In [33]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

# Model 2: LSTM Model

In [34]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [35]:
model_2.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [36]:
model_2_history = model_2.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")]
)

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220704-095030
Epoch 1/5
215/215 [==============================] - 13s 19ms/step - loss: 0.2260 - accuracy: 0.9237 - val_loss: 0.5569 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1601 - accuracy: 0.9403 - val_loss: 0.6185 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1278 - accuracy: 0.9514 - val_loss: 0.7210 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1078 - accuracy: 0.9587 - val_loss: 0.8361 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0826 - accuracy: 0.9661 - val_loss: 1.0495 - val_accuracy: 0.7677


In [37]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[3.0720353e-02],
       [9.5613015e-01],
       [9.9997926e-01],
       [1.3120647e-01],
       [1.6340890e-04],
       [9.9995673e-01],
       [9.8629308e-01],
       [9.9999416e-01],
       [9.9999011e-01],
       [3.3258006e-01]], dtype=float32)

In [38]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [39]:
model_2_results = calculate_results(y_true=val_labels, y_pred=model_2_preds)
model_2_results

{'accuracy': 76.77165354330708,
 'f1': 0.766597011860758,
 'precision': 0.7677367335034279,
 'recall': 0.7677165354330708}

# Model 3: GRU Model

In [40]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.GRU(64, return_sequences=True)(x)
#x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [41]:
model_3.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [42]:
model_3_history = model_3.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_GRU")]
)

Saving TensorBoard log files to: model_logs/model_2_GRU/20220704-095056
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.1583 - accuracy: 0.9402 - val_loss: 0.7340 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0842 - accuracy: 0.9672 - val_loss: 1.0315 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0617 - accuracy: 0.9746 - val_loss: 0.9795 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0557 - accuracy: 0.9762 - val_loss: 1.5324 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0507 - accuracy: 0.9772 - val_loss: 1.2313 - val_accuracy: 0.7598


In [43]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[5.2854151e-01],
       [8.5362434e-01],
       [9.9998379e-01],
       [1.9301133e-01],
       [1.3022892e-05],
       [9.9982673e-01],
       [9.6794879e-01],
       [9.9998569e-01],
       [9.9998367e-01],
       [4.8223981e-01]], dtype=float32)

In [44]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_3_results = calculate_results(y_true=val_labels, y_pred=model_3_preds)
model_3_results

{'accuracy': 75.98425196850394,
 'f1': 0.7590768821741806,
 'precision': 0.7594780880165999,
 'recall': 0.7598425196850394}

In [46]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

# Model 4: Bi-Directional RNN Model

In [47]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [48]:
model_4.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [49]:
model_4_history = model_4.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_4_GRU")]
)

Saving TensorBoard log files to: model_logs/model_4_GRU/20220704-095107
Epoch 1/5
215/215 [==============================] - 6s 12ms/step - loss: 0.1025 - accuracy: 0.9672 - val_loss: 1.3786 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0524 - accuracy: 0.9766 - val_loss: 1.4869 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0430 - accuracy: 0.9794 - val_loss: 1.5265 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0398 - accuracy: 0.9794 - val_loss: 1.6599 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0404 - accuracy: 0.9800 - val_loss: 1.4017 - val_accuracy: 0.7664


In [50]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[5.4259808e-03],
       [4.8201060e-01],
       [9.9989271e-01],
       [3.3470914e-01],
       [5.7599764e-06],
       [9.9998355e-01],
       [7.6947665e-01],
       [9.9999845e-01],
       [9.9998951e-01],
       [9.9375463e-01]], dtype=float32)

In [51]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [52]:
model_4_results = calculate_results(y_true=val_labels, y_pred=model_4_preds)
model_4_results

{'accuracy': 76.64041994750657,
 'f1': 0.7642857598265804,
 'precision': 0.7679094748917975,
 'recall': 0.7664041994750657}

# Model 5: Conv1D Model

In [56]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")

x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [57]:
model_5.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_7 (Dense)             (None, 1)              

In [71]:
model_5_history = model_5.fit(
    train_sentences,
    train_labels,
    epochs=10,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "Conv1D")]
)

Saving TensorBoard log files to: model_logs/Conv1D/20220704-100959
Epoch 1/10
215/215 [==============================] - 2s 10ms/step - loss: 0.0336 - accuracy: 0.9822 - val_loss: 1.2349 - val_accuracy: 0.7441
Epoch 2/10
215/215 [==============================] - 1s 6ms/step - loss: 0.0335 - accuracy: 0.9828 - val_loss: 1.2153 - val_accuracy: 0.7428
Epoch 3/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0327 - accuracy: 0.9829 - val_loss: 1.2355 - val_accuracy: 0.7493
Epoch 4/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0313 - accuracy: 0.9844 - val_loss: 1.2231 - val_accuracy: 0.7428
Epoch 5/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0324 - accuracy: 0.9829 - val_loss: 1.2238 - val_accuracy: 0.7467
Epoch 6/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0321 - accuracy: 0.9818 - val_loss: 1.1795 - val_accuracy: 0.7546
Epoch 7/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0306

In [72]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[3.5176195e-02],
       [9.8928589e-01],
       [9.9998474e-01],
       [2.0209480e-02],
       [2.1215921e-05],
       [9.9984980e-01],
       [9.5722198e-01],
       [9.9999666e-01],
       [9.9992955e-01],
       [1.0709109e-01]], dtype=float32)

In [73]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [74]:
model_5_results = calculate_results(y_true=val_labels, y_pred=model_5_preds)
model_5_results

{'accuracy': 74.93438320209974,
 'f1': 0.7487274088908836,
 'precision': 0.7488632680071313,
 'recall': 0.7493438320209974}